In [1]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [14]:
from data import Data
data = Data()
# data.filter_no_covid_cases()
# data.keep_15_symptoms()
# data.filter_low_data_regions(threshold=0.15)
data.fill_na(0)
dat = data.d
print(dat['open_covid_region_code'].unique())

['US-AK' 'US-DC' 'US-DE' 'US-HI' 'US-ID' 'US-ME' 'US-MT' 'US-ND' 'US-NE'
 'US-NH' 'US-NM' 'US-RI' 'US-SD' 'US-VT' 'US-WV' 'US-WY']


c:\users\julian\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
def region_validation_splits(d, n_splits=5):
    regions = d["open_covid_region_code"].unique()
    kf = KFold(n_splits=n_splits)
    for train_i, val_i in kf.split(regions):
        train = regions[train_i]
        val = regions[val_i]
        dat_train = d[d["open_covid_region_code"].isin(train)]
        dat_val = d[d["open_covid_region_code"].isin(val)]
        x_train = dat_train.iloc[:, 6:-1]
        y_train = dat_train.iloc[:, -1]
        x_val = dat_val.iloc[:, 6:-1]
        y_val = dat_val.iloc[:, -1]
        yield x_train, y_train, x_val, y_val

In [4]:
def time_validation_split(d, split_date):
    dat_train = d[d["date"] <= split_date]
    dat_val = d[d["date"] > split_date]
    x_train = dat_train.iloc[:, 6:-1]
    y_train = dat_train.iloc[:, -1]
    x_val = dat_val.iloc[:, 6:-1]
    y_val = dat_val.iloc[:, -1]
    yield x_train, y_train, x_val, y_val

In [5]:
def calc_MSE(model, validation_sets):
    squared_error = 0
    n_validations = 0
    for x_train, y_train, x_val, y_val in validation_sets:
        model.fit(x_train, y_train)
        pred = model.predict(x_val)
        squared_error += sum((pred - y_val) ** 2)
        n_validations += x_val.shape[0]
    return squared_error / n_validations

In [18]:
n_neighbors = np.zeros(100)
for i in range(n_neighbors.shape[0]):
    neigh = KNeighborsRegressor(n_neighbors=i+1)
    n_neighbors[i] = calc_MSE(neigh, region_validation_splits(dat, n_splits=5))
best = np.argmin(n_neighbors) + 1
print(f"best K = {best}, MSE = {n_neighbors[best - 1]}")

best K = 81, MSE = 4466.252648224357


In [17]:
leaf_samples = np.zeros(100)
for i in range(leaf_samples.shape[0]):
    tree = DecisionTreeRegressor(min_samples_leaf=i+1)
    leaf_samples[i] = calc_MSE(tree, region_validation_splits(dat, n_splits=5))
best = np.argmin(leaf_samples) + 1
print(f"best min_samples_leaf = {best}, MSE = {leaf_samples[best - 1]}")

best min_samples_leaf = 45, MSE = 4080.3749903676476


In [16]:
n_neighbors = np.zeros(100)
for i in range(n_neighbors.shape[0]):
    neigh = KNeighborsRegressor(n_neighbors=i+1)
    n_neighbors[i] = calc_MSE(neigh, time_validation_split(dat, "2020-08-10"))
best = np.argmin(n_neighbors) + 1
print(f"best K = {best}, MSE = {n_neighbors[best - 1]}")

best K = 43, MSE = 1178.432778979629


In [15]:
leaf_samples = np.zeros(100)
for i in range(leaf_samples.shape[0]):
    tree = DecisionTreeRegressor(min_samples_leaf=i+1)
    leaf_samples[i] = calc_MSE(tree, time_validation_split(dat, "2020-08-10"))
best = np.argmin(leaf_samples) + 1
print(f"best min_samples_leaf = {best}, MSE = {leaf_samples[best - 1]}")

best min_samples_leaf = 30, MSE = 1234.4315375578458
